In [1]:
%load_ext cython
%load_ext fortranmagic

In [2]:
import numpy as np
from scipy.misc import electrocardiogram

In [3]:
x1d = electrocardiogram()
x3d = x1d.reshape((1, -1, 1))

In [18]:
%%cython
# cython: infer_types = True
# cython: boundscheck = False
# cython: wraparound = False
cimport cython
from numpy import zeros, double as npy_double, intc, nanmin, nanmax, array
from libc.math cimport log, exp, ceil, sqrt, isnan


cdef void mean_sd_1d(const double[:] x, double* mean, double* std):
    cdef Py_ssize_t n = x.size, i

    cdef double k = x[0]
    cdef double Ex = 0., Ex2 = 0.
    mean[0] = 0.

    for i in range(n):
        mean[0] += x[i]
        Ex += x[i] - k
        Ex2 += (x[i] - k)**2
    
    std[0] = sqrt((Ex2 - (Ex**2 / n)) / (n - 1))
    mean[0] /= n
    
    
cpdef hist(const double[:] signal, int ncells, double min_val, double max_val, Py_ssize_t N):
    cdef Py_ssize_t i
    counts = zeros(ncells, dtype=intc)

    cdef int[:] c_view = counts
    cdef int idx
    cdef double bin_width = (max_val - min_val) / <double>(ncells)

    if bin_width == 0.0:
        bin_width = 1.0  # prevent 0 division
    
    for i in range(N):
        if isnan(signal[i]):
            continue
        
        idx = <int>((signal[i] - min_val) / bin_width)
        if idx == ncells:
            idx -= 1
        
        c_view[idx] += 1
    
    return counts


cpdef histogram(const double[:] signal, double[:] descriptor):
    cdef Py_ssize_t N = signal.size
    cdef double min_val = nanmin(signal)
    cdef double max_val = nanmax(signal)
    cdef double delta = (max_val - min_val) / <double>(N - 1)

    descriptor[0] = min_val - delta / 2
    descriptor[1] = max_val + delta / 2
    descriptor[2] = ceil(sqrt(N))

    return hist(signal, <int>(descriptor[2]), min_val, max_val, N)


def SignalEntropy(const double[:, :, :] signal):
    cdef Py_ssize_t M = signal.shape[0], N = signal.shape[1], P = signal.shape[2]

    res = zeros((M, P), dtype=npy_double)

    cdef double[:, ::1] result = res
    cdef double[::1] d = zeros(3, dtype=npy_double)
    cdef double[::1] data_norm = zeros(N, dtype=npy_double)

    cdef double logf, nbias, count, estimate, h_n, std, mean
    cdef Py_ssize_t i, j, k, n

    for i in range(M):
        for k in range(P):
            std = 0.
            mean = 0.
            mean_sd_1d(signal[i, :, k], &mean, &std)
            
            if std == 0:
                std = 1.  # ensure no division by 0
            for j in range(N):
                data_norm[j] = signal[i, j, k] / std
            
            h = histogram(data_norm, d)

            if (d[0] == d[1]):  # data is constant
                result[i] = 0.0  # no information
                continue
        
            count = 0
            estimate = 0

            for n in range(<int>(d[2])):
                h_n = h[n]
                if h_n > 0:
                    logf = log(h_n)
                else:
                    logf = 0.0
            
                count += h_n
                estimate -= h_n * logf
            

            nbias = -(d[2] - 1) / (2 * count)
            estimate = estimate / count + log(count) + log((d[1] - d[0]) / d[2]) - nbias

            result[i, k] = exp(estimate**2) - 1 - 1

    return res


def SignalEntropy2(const double[:, :, :] signal):
    cdef Py_ssize_t M = signal.shape[0], N = signal.shape[1], P = signal.shape[2]

    res = zeros((M, N), dtype=npy_double)

    cdef double[:, ::1] result = res
    cdef double[::1] d = zeros(3, dtype=npy_double)
    cdef double[::1] data_norm = zeros(P, dtype=npy_double)

    cdef double logf, nbias, count, estimate, h_n, std, mean
    cdef Py_ssize_t i, j, k, n

    for i in range(M):
        for j in range(N):
            std = 0.
            mean = 0.
            mean_sd_1d(signal[i, j, :], &mean, &std)
            
            if std == 0:
                std = 1.  # ensure no division by 0
            
            h = histogram(signal[i, j, :], d)
            d[0] /= std
            d[1] /= std

            if (d[0] == d[1]):  # data is constant
                result[i] = 0.0  # no information
                continue
        
            count = 0
            estimate = 0

            for n in range(<int>(d[2])):
                h_n = h[n]
                if h_n > 0:
                    logf = log(h_n)
                else:
                    logf = 0.0
            
                count += h_n
                estimate -= h_n * logf
            

            nbias = -(d[2] - 1) / (2 * count)
            estimate = estimate / count + log(count) + log((d[1] - d[0]) / d[2]) - nbias
            result[i, j] = exp(estimate**2) - 1 - 1

    return res

In [5]:
from sigent import hist as fhist, histogram as fhistogram, signalentropy as fsignalentropy

In [10]:
np.random.seed(5)
x = np.random.rand(50000, 150, 3)
xc = np.ascontiguousarray(x.transpose([0, 2, 1]))
xf = np.asfortranarray(x.transpose([1, 2, 0]))

In [11]:
%timeit SignalEntropy(x)

7.76 s ± 78.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%timeit -n 1 -r 1 SignalEntropy2(xc)

8.18 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [13]:
%timeit np.asfortranarray(x.transpose([1, 2, 0]))
%timeit fsignalentropy(xf)

35.2 ms ± 14.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
138 ms ± 3.58 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [14]:
np.allclose(SignalEntropy(x[:5000]), fsignalentropy(xf[:, :, :5000]).T)

True

In [16]:
np.allclose(SignalEntropy(x[:5000]), SignalEntropy2(xc[:5000]))

True

In [38]:
fsignalentropy(xf).T

array([[-0.99895339],
       [-0.99965671],
       [-0.99998348],
       [-0.99944761],
       [-0.9998934 ]])

In [37]:
xf.T.shape

(5, 1, 150)

In [ ]:
%%fortran

subroutine fmean_sd_1d(n, x, mn, sd)
    implicit none
    integer(8), intent(in) :: n
    real(8), intent(in) :: x(n)
    real(8), intent(out) :: mn, sd
!f2py intent(hide) :: n
    real(8) :: k, Ex, Ex2
    integer(8) :: i
    
    Ex2 = 0._8
    mn = 0.0_8
    k = x(1)
    
    do i=1, n
        mn = mn + x(i)
        Ex2 = Ex2 + (x(i) - k)**2
    end do
    ! can compute outside of loop since mean addition is the same
    Ex = mn - (n * k)
    
    sd = sqrt((Ex2 - (Ex**2 / n)) / (n - 1))
    mn = mn / n
end subroutine

subroutine fhist(n, signal, ncells, min_val, max_val, counts)
    implicit none
    integer(8), intent(in) :: n, ncells
    real(8), intent(in) :: signal(n), min_val, max_val
    integer(8), intent(out) :: counts(ncells)
!f2py intent(hide) :: n
    integer(8) :: i, idx
    real(8) :: bin_width

    counts = 0_8
    
    bin_width = (max_val - min_val) / ncells
            
    if (bin_width .EQ. 0.0_8) then
        bin_width = 1.0_8  ! prevent 0 division
    end if
    
    do i=1, n
        if (.NOT. isnan(signal(i))) then
            idx = int((signal(i) - min_val) / bin_width, 8) + 1_8
            if (idx > ncells) then
                idx = ncells
            end if
            
            counts(idx) = counts(idx) + 1
        end if
    end do
end subroutine

subroutine fhistogram(n, k, signal, descriptor, counts)
    ! k is ceiling(sqrt(n))
    implicit none
    integer(8), intent(in) :: n, k
    real(8), intent(in) :: signal(n)
    real(8), intent(inout) :: descriptor(3)
    integer(8), intent(out) :: counts(k)
!f2py intent(hide) :: n
    real(8) :: min_val, max_val, delta
    
    min_val = minval(signal)
    max_val = maxval(signal)
    delta = (max_val - min_val) / (n - 1)
    
    descriptor(1) = min_val - delta / 2
    descriptor(2) = max_val + delta / 2
    descriptor(3) = real(k, 8)
    
    call fhist(n, signal, k, min_val, max_val, counts)
end subroutine
    

subroutine fsignalEntropy(p, n, m, signal, sigEnt)
    implicit none
    integer(8), intent(in) :: p, n, m
    real(8), intent(in) :: signal(p, n, m)
    real(8), intent(out) :: sigEnt(p, m)
!fp2y intent(hide) :: p, n, m
    real(8) :: d(3), data_norm(n), logf, nbias, count
    real(8) :: estimate, std, mean
    integer(8) :: i, j, k, sqn
    integer(8) :: h(ceiling(sqrt(real(n))))
            
    sqn = ceiling(sqrt(real(n)))
    
    do k=1, m
        do i=1, p
            std = 0._8
            mean = 0._8
            call fmean_sd_1d(n, signal(i, :, k), mean, std)
            
            if (std == 0._8) then
                std = 1._8  ! ensure no division by 0
            end if
            
            data_norm = signal(i, :, k) / std
            
            call fhistogram(n, sqn, data_norm, d, h)
            
            if (d(1) == d(2)) then
                sigEnt(i, k) = 0._8
            else
                count = 0._8
                estimate = 0._8

                do j=1, int(d(3), 8)
                    if (h(j) > 0) then
                        logf = log(real(h(j), 8))
                    else
                        logf = 0._8
                    end if

                    count = count + h(j)
                    estimate = estimate - h(j) * logf
                end do

                nbias = -(d(3) - 1) / (2 * count)
                estimate = estimate / count + log(count) + log((d(2) - d(1)) / d(3)) - nbias
                sigEnt(i, k) = exp(estimate**2) - 2
            end if
        end do
    end do
end subroutine


subroutine fsignalEntropy2(m, n, p, signal, sigEnt)
    ! # m is the signal dimension
    ! # n is the axis dimension
    ! # p is the window dimension
    implicit none
    integer(8), intent(in) :: m, n, p
    real(8), intent(in) :: signal(m, n, p)
    real(8), intent(out) :: sigEnt(n, p)
!fp2y intent(hide) :: m, n, p
    real(8) :: d(3), data_norm(m), nbias, count
    real(8) :: estimate, std, mean
    real(8) :: logf
    integer(8) :: i, j, k, sqn
    integer(8) :: h(ceiling(sqrt(real(m))))
            
    sqn = ceiling(sqrt(real(m)))
    
    do k=1, p
        do j=1, n
            std = 0._8
            mean = 0._8
            call fmean_sd_1d(m, signal(:, j, k), mean, std)
            
            if (std == 0._8) then
                std = 1._8  ! ensure no division by 0
            end if
            
#             data_norm = signal(:, j, k) / std
            
            call fhistogram(m, sqn, signal(:, j, k), d, h)
            
            if (d(1) == d(2)) then
                sigEnt(j, k) = 0._8
            else
                count = 0._8
                estimate = 0._8
                
                do i=1, int(d(3), 8)
                    if (h(i) > 0) then
                        logf = log(real(h(i), 8))
                    else
                        logf = 0._8
                    end if

                    count = count + h(i)
                    estimate = estimate - h(i) * logf
                end do

                nbias = -(d(3) - 1) / (2 * count)
                estimate = estimate / count + log(count) + log((d(2) - d(1)) / d(3)) - nbias
                sigEnt(j, k) = exp(estimate**2) - 2
            end if
        end do
    end do
end subroutine

subroutine fsignalEntropy3(m, n, p, signal, sigEnt)
    ! # m is the signal dimension
    ! # n is the axis dimension
    ! # p is the window dimension
    implicit none
    integer(8), intent(in) :: m, n, p
    real(8), intent(in) :: signal(m, n, p)
    real(8), intent(out) :: sigEnt(n, p)
!fp2y intent(hide) :: m, n, p
    real(8) :: d(3), data_norm(m), nbias, count
    real(8) :: estimate, std, mean
    real(8) :: logf
    integer(8) :: i, j, k, sqn
    integer(8) :: h(ceiling(sqrt(real(m))))
            
    sqn = ceiling(sqrt(real(m)))
    
    do k=1, p
        do j=1, n
            std = 0._8
            mean = 0._8
            call fmean_sd_1d(m, signal(:, j, k), mean, std)
            
            if (std == 0._8) then
                std = 1._8  ! ensure no division by 0
            end if
            
#             data_norm = signal(:, j, k) / std
            
            call fhistogram(m, sqn, signal(:, j, k), d, h)
            
            if (d(1) == d(2)) then
                sigEnt(j, k) = 0._8
            else
                count = 0._8
                estimate = 0._8
                
                do i=1, int(d(3), 8)
                    if (h(i) > 0) then
                        logf = log(real(h(i), 8))
                    else
                        logf = 0._8
                    end if

                    count = count + h(i)
                    estimate = estimate - h(i) * logf
                end do

                nbias = -(d(3) - 1) / (2 * count)
                estimate = estimate / count + log(count) + log((d(2) - d(1)) / d(3)) - nbias
                sigEnt(j, k) = exp(estimate**2) - 2
            end if
        end do
    end do
end subroutine
    

In [ ]:
%timeit fmean_sd_1d(x1d)
%timeit fmean_sd_1d_2(x1d)

In [ ]:
%timeit np.mean(x1d); np.std(x1d)

In [ ]:
mn, mx, n = x1d.min(), x1d.max(), x1d.size
k = int(np.ceil(np.sqrt(n)))
d = np.zeros(3)

In [ ]:
%timeit hist(x1d, 10, mn, mx, n)
%timeit fhist(x1d, 10, mn, mx)

In [ ]:
%timeit histogram(x1d, d)
%timeit fhistogram(k, x1d, d)

In [ ]:
%timeit SignalEntropy(x3d)
%timeit fsignalentropy(x3d.T)

In [ ]:
fsignalentropy(x3d.T)

In [ ]:
SignalEntropy(x3d)

In [ ]:
xb = np.random.rand(50000, 150, 3)
xc = np.ascontiguousarray(xb.transpose([0, 2, 1]))

In [ ]:
%timeit fsignalentropy(xb.T)
%timeit fsignalentropy2(xc.T)
%timeit fsignalentropy3(xc.T)

In [ ]:
%timeit -n 1 -r 1 SignalEntropy(xb)

In [ ]:
cres = SignalEntropy(xb)

In [ ]:
np.allclose(fsignalentropy(xb.T).T, cres)

In [ ]:
np.allclose(fsignalentropy3(xc.T).T, cres)

In [ ]:
c = histogram(x1d, d)
c1 = histogram(x1d / np.std(x1d), d)
np.allclose(c, c1)

In [ ]:
%%fortran

subroutine test1(n, x, ret, ret1, t, cr)
    implicit none
    integer(8), intent(in) :: n
    real(8), intent(in) :: x(n)
    real(8), intent(out) :: ret, ret1
    integer(8), intent(out) :: t, cr
!f2py intent(hide) :: n
    integer(8) :: c1, c2, cm
    
    call system_clock(c1, cr, cm)
    ret = sum(x) - (n * x(1))
    ret1 = sum((x-x(1))**2) / n
    call system_clock(c2, cr, cm)
    t = c2 - c1
end subroutine

subroutine test2(n, x, ret, ret1, t, cr)
    implicit none
    integer(8), intent(in) :: n
    real(8), intent(in) :: x(n)
    real(8), intent(out) :: ret, ret1
    integer(8), intent(out) :: t, cr
!f2py intent(hide) :: n
    integer(8) :: i, c1, c2, cm
    
    call system_clock(c1, cr, cm)
    
    ret = 0._8
    do i=1, n
        ret = ret + (x(i) - x(1))
        ret1 = ret1 + (x(i) - x(1))**2
    end do
    ret1 = ret1 / n
    
    call system_clock(c2, cr, cm)
    t = c2 - c1
end subroutine

In [ ]:
z = np.random.rand(5000)

In [ ]:
%timeit test1(z)
%timeit test2(z)

In [ ]:
t1 = []
t2 = []
for i in range(100000):
    _, _, t1_, cr1 = test1(z)
    _, _, t2_, cr2 = test2(z)
    
    t1.append(t1_ / cr1)
    t2.append(t2_ / cr2)

print(np.mean(t1), np.std(t1))
print(np.mean(t2), np.std(t2))

In [ ]:
%%fortran

subroutine fmean_sd_1d(n, x, mn, sd)
    implicit none
    integer(8), intent(in) :: n
    real(8), intent(in) :: x(n)
    real(8), intent(out) :: mn, sd
!f2py intent(hide) :: n, mn, sd
    real(8) :: k, Ex, Ex2
    integer(8) :: i
    
    Ex2 = 0._8
    mn = 0.0_8
    k = x(1)
    
    do i=1, n
        mn = mn + x(i)
        Ex2 = Ex2 + (x(i) - k)**2
    end do
    Ex = mn - (n * k)  ! account for not subtracting in the loop
    
    sd = sqrt((Ex2 - (Ex**2 / n)) / (n - 1))
    mn = mn / n
end subroutine


subroutine mean_sd_3d(m, n, p, x, mn, sd)
    ! # m is the signal dimension
    ! # n is the axis dimension
    ! # p is the window dimension
    implicit none
    integer(8), intent(in) :: m, n, p
    real(8), intent(in) :: x(m, n, p)
    real(8), intent(out) :: mn(n, p), sd(n, p)
!f2py intent(hide) :: m, n, p
    integer(8) :: i, j, k
    real(8) :: Ex, Ex2
            
    mn = 0._8
            
    do k=1, p
        do j=1, n
            Ex = 0._8
            Ex2 = 0._8
            do i=1, m
                mn(j, k) = mn(j, k) + x(i, j, k)
                Ex2 = Ex2 + (x(i, j, k) - x(1, j, k))**2
            end do
            Ex = mn(j, k) - (m * x(1, j, k))
            sd(j, k) = sqrt((Ex2 - (Ex**2 / m)) / (m - 1))
        end do
    end do
    mn = mn / m
end subroutine

subroutine test_mean(m, n, p, x, mn, sd)
    ! # m is the signal dimension
    ! # n is the axis dimension
    ! # p is the window dimension
    implicit none
    integer(8), intent(in) :: m, n, p
    real(8), intent(in) :: x(m, n, p)
    real(8), intent(out) :: mn(n, p), sd(n, p)
!f2py intent(hide) :: m, n, p
    integer(8) :: i, j, k
    
    do k=1, p
        do j=1, n
            call fmean_sd_1d(m, x(:, j, k), mn(j, k), sd(j, k))
        end do
    end do
end subroutine

In [ ]:
%timeit mean_sd_3d(xc.T)
%timeit test_mean(xc.T)

In [ ]:
np.allclose(mean_sd_3d(xc.T), test_mean(xc.T))